# OTEX Regional Analysis

This notebook demonstrates how to analyze OTEC potential for a specific geographic region using real oceanographic data.

## Contents
1. [Setup](#1.-Setup)
2. [Region Selection](#2.-Region-Selection)
3. [Running Analysis](#3.-Running-Analysis)
4. [Results Exploration](#4.-Results-Exploration)
5. [Visualization](#5.-Visualization)
6. [Uncertainty at Best Site](#6.-Uncertainty-at-Best-Site)

## Prerequisites

- CMEMS credentials configured (see installation guide)
- Internet connection for data download

In [ ]:
# Import libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Plot settings
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

print("Libraries loaded successfully!")

## 1. Setup

Import OTEX data utilities. No path configuration needed when OTEX is installed via pip.

In [ ]:
# Load bundled region and site data from the otex package
from otex.data import load_regions, load_sites

regions_df = load_regions()
sites_all = load_sites()

print(f"Regions database: {len(regions_df)} regions loaded")
print(f"Sites database: {len(sites_all)} sites loaded")

## 2. Region Selection

Explore available regions and select one for analysis.

In [ ]:
# Load available regions (already loaded above)
print(f"Total regions available: {len(regions_df)}")
print(f"\nFirst 20 regions:")
regions_df.head(20)

In [ ]:
# Search for specific regions
search_term = 'Jamaica'  # Change this to search for other regions
matching = regions_df[regions_df['region'].str.contains(search_term, case=False)]
print(f"Regions matching '{search_term}':")
matching

In [ ]:
# Select region for analysis
REGION = 'Jamaica'

# Get region bounds
region_info = regions_df[regions_df['region'] == REGION].iloc[0]
print(f"Selected Region: {REGION}")
print(f"=" * 40)
print(f"Bounds:")
print(f"  North: {region_info['north']:.3f}°")
print(f"  South: {region_info['south']:.3f}°")
print(f"  East:  {region_info['east']:.3f}°")
print(f"  West:  {region_info['west']:.3f}°")
print(f"  Demand: {region_info['demand']} TWh/year")

In [ ]:
# Explore potential OTEC sites in the region
sites_df = sites_all.copy()
sites_df.columns = ['id', 'longitude', 'latitude', 'region', 'water_depth', 'dist_shore']

# Filter for selected region
region_sites = sites_df[sites_df['region'] == REGION].copy()

print(f"\nPotential OTEC Sites in {REGION}")
print(f"=" * 40)
print(f"Total sites: {len(region_sites)}")
print(f"\nWater Depth (m):")
print(f"  Min:  {region_sites['water_depth'].max():.0f}")  # Note: depths are negative
print(f"  Max:  {region_sites['water_depth'].min():.0f}")
print(f"  Mean: {region_sites['water_depth'].mean():.0f}")
print(f"\nDistance to Shore (km):")
print(f"  Min:  {region_sites['dist_shore'].min():.1f}")
print(f"  Max:  {region_sites['dist_shore'].max():.1f}")
print(f"  Mean: {region_sites['dist_shore'].mean():.1f}")

In [ ]:
# Visualize site locations
fig, ax = plt.subplots(figsize=(10, 8))

scatter = ax.scatter(
    region_sites['longitude'],
    region_sites['latitude'],
    c=-region_sites['water_depth'],  # Positive depth for color
    cmap='viridis',
    s=50,
    alpha=0.7
)

plt.colorbar(scatter, label='Water Depth (m)')
ax.set_xlabel('Longitude', fontsize=11)
ax.set_ylabel('Latitude', fontsize=11)
ax.set_title(f'Potential OTEC Sites in {REGION}', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Running Analysis

Run the regional OTEC analysis. This will:
1. Download CMEMS temperature data (if not cached)
2. Process temperature profiles
3. Size OTEC plants for each site
4. Calculate LCOE

**Note:** First run may take 10-20 minutes for data download.

In [ ]:
# Analysis parameters
YEAR = 2020
P_GROSS = -50000  # 50 MW
COST_LEVEL = 'low_cost'
CYCLE_TYPE = 'rankine_closed'
FLUID_TYPE = 'ammonia'

print(f"Analysis Configuration")
print(f"=" * 40)
print(f"Region: {REGION}")
print(f"Year: {YEAR}")
print(f"Plant Size: {-P_GROSS/1000:.0f} MW")
print(f"Cost Level: {COST_LEVEL}")
print(f"Cycle: {CYCLE_TYPE}")
print(f"Fluid: {FLUID_TYPE}")

In [ ]:
# Import and run regional analysis from the otex package
from otex.regional import run_regional_analysis

print(f"\nStarting regional analysis for {REGION}...")
print(f"This may take several minutes if downloading data for the first time.\n")

otec_plants, sites_analyzed = run_regional_analysis(
    studied_region=REGION,
    p_gross=P_GROSS,
    cost_level=COST_LEVEL,
    year=YEAR,
    cycle_type=CYCLE_TYPE,
    fluid_type=FLUID_TYPE,
    use_coolprop=True
)

## 4. Results Exploration

Load and explore the analysis results.

In [ ]:
# Find the results file
results_dir = Path(f'Data_Results/{REGION}/{REGION}_{YEAR}_{-P_GROSS/1000}_MW_{COST_LEVEL}'.replace(' ', '_'))
results_file = list(results_dir.glob('OTEC_sites_*.csv'))[0]

print(f"Results directory: {results_dir}")
print(f"Results file: {results_file.name}")

# Load results
results = pd.read_csv(results_file, sep=';', index_col='id')
print(f"\nLoaded {len(results)} sites")
results.head()

In [ ]:
# Summary statistics
print(f"\nResults Summary for {REGION}")
print(f"=" * 50)
print(f"\nFeasible Sites: {len(results)}")

print(f"\nLCOE (ct/kWh):")
print(f"  Min:    {results['LCOE'].min():.2f}")
print(f"  Max:    {results['LCOE'].max():.2f}")
print(f"  Mean:   {results['LCOE'].mean():.2f}")
print(f"  Median: {results['LCOE'].median():.2f}")

print(f"\nNet Power (MW):")
print(f"  Min:    {results['p_net_nom'].min():.1f}")
print(f"  Max:    {results['p_net_nom'].max():.1f}")
print(f"  Mean:   {results['p_net_nom'].mean():.1f}")

print(f"\nAnnual Energy Production (GWh):")
print(f"  Total:  {results['AEP'].sum():.1f}")
print(f"  Mean:   {results['AEP'].mean():.1f}")

print(f"\nCAPEX ($M):")
print(f"  Min:    {results['CAPEX'].min():.1f}")
print(f"  Max:    {results['CAPEX'].max():.1f}")
print(f"  Mean:   {results['CAPEX'].mean():.1f}")

In [ ]:
# Best sites by LCOE
print(f"\nTop 10 Sites by LCOE")
print(f"=" * 70)

best_sites = results.nsmallest(10, 'LCOE')[['longitude', 'latitude', 'LCOE', 'p_net_nom', 'AEP', 'CAPEX']]
best_sites

In [ ]:
# Temperature conditions at sites
print(f"\nTemperature Conditions")
print(f"=" * 50)

print(f"\nWarm Water Temperature (°C):")
print(f"  Min median: {results['T_WW_med'].min():.1f}")
print(f"  Max median: {results['T_WW_med'].max():.1f}")
print(f"  Mean median: {results['T_WW_med'].mean():.1f}")

print(f"\nCold Water Temperature (°C):")
print(f"  Min median: {results['T_CW_med'].min():.1f}")
print(f"  Max median: {results['T_CW_med'].max():.1f}")
print(f"  Mean median: {results['T_CW_med'].mean():.1f}")

# Calculate temperature difference
results['delta_T'] = results['T_WW_med'] - results['T_CW_med']
print(f"\nTemperature Difference (°C):")
print(f"  Min: {results['delta_T'].min():.1f}")
print(f"  Max: {results['delta_T'].max():.1f}")
print(f"  Mean: {results['delta_T'].mean():.1f}")

## 5. Visualization

In [ ]:
# LCOE map
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# LCOE map
scatter = axes[0].scatter(
    results['longitude'],
    results['latitude'],
    c=results['LCOE'],
    cmap='RdYlGn_r',
    s=80,
    alpha=0.8,
    edgecolor='white',
    linewidth=0.5
)
plt.colorbar(scatter, ax=axes[0], label='LCOE (ct/kWh)')
axes[0].set_xlabel('Longitude', fontsize=11)
axes[0].set_ylabel('Latitude', fontsize=11)
axes[0].set_title(f'LCOE Map - {REGION}', fontsize=14, fontweight='bold')

# Highlight best site
best = results.loc[results['LCOE'].idxmin()]
axes[0].scatter(best['longitude'], best['latitude'], c='blue', s=200, marker='*', 
                edgecolor='white', linewidth=2, label=f"Best: {best['LCOE']:.1f} ct/kWh")
axes[0].legend(loc='upper right')

# Net Power map
scatter2 = axes[1].scatter(
    results['longitude'],
    results['latitude'],
    c=results['p_net_nom'],
    cmap='Blues',
    s=80,
    alpha=0.8,
    edgecolor='white',
    linewidth=0.5
)
plt.colorbar(scatter2, ax=axes[1], label='Net Power (MW)')
axes[1].set_xlabel('Longitude', fontsize=11)
axes[1].set_ylabel('Latitude', fontsize=11)
axes[1].set_title(f'Net Power Map - {REGION}', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig(f'{REGION}_maps.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Distribution plots
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# LCOE histogram
axes[0, 0].hist(results['LCOE'], bins=25, color='steelblue', edgecolor='white', alpha=0.7)
axes[0, 0].axvline(results['LCOE'].median(), color='red', linestyle='--', label=f"Median: {results['LCOE'].median():.1f}")
axes[0, 0].set_xlabel('LCOE (ct/kWh)')
axes[0, 0].set_ylabel('Number of Sites')
axes[0, 0].set_title('LCOE Distribution')
axes[0, 0].legend()

# LCOE vs Temperature Difference
axes[0, 1].scatter(results['delta_T'], results['LCOE'], alpha=0.6, c='steelblue')
z = np.polyfit(results['delta_T'], results['LCOE'], 1)
p = np.poly1d(z)
x_line = np.linspace(results['delta_T'].min(), results['delta_T'].max(), 100)
axes[0, 1].plot(x_line, p(x_line), 'r--', linewidth=2, label='Trend')
axes[0, 1].set_xlabel('Temperature Difference (°C)')
axes[0, 1].set_ylabel('LCOE (ct/kWh)')
axes[0, 1].set_title('LCOE vs Temperature Difference')
axes[0, 1].legend()

# Net Power histogram
axes[1, 0].hist(results['p_net_nom'], bins=25, color='forestgreen', edgecolor='white', alpha=0.7)
axes[1, 0].axvline(results['p_net_nom'].median(), color='red', linestyle='--', 
                   label=f"Median: {results['p_net_nom'].median():.1f} MW")
axes[1, 0].set_xlabel('Net Power (MW)')
axes[1, 0].set_ylabel('Number of Sites')
axes[1, 0].set_title('Net Power Distribution')
axes[1, 0].legend()

# CAPEX vs Net Power
axes[1, 1].scatter(results['p_net_nom'], results['CAPEX'], alpha=0.6, c='purple')
axes[1, 1].set_xlabel('Net Power (MW)')
axes[1, 1].set_ylabel('CAPEX ($M)')
axes[1, 1].set_title('CAPEX vs Net Power')

plt.tight_layout()
plt.savefig(f'{REGION}_distributions.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Uncertainty at Best Site

Run uncertainty analysis for the best site identified.

In [ ]:
# Get best site conditions
best_site = results.loc[results['LCOE'].idxmin()]

print(f"Best Site Analysis")
print(f"=" * 40)
print(f"Location: ({best_site['longitude']:.3f}, {best_site['latitude']:.3f})")
print(f"T_WW (median): {best_site['T_WW_med']:.1f} °C")
print(f"T_CW (median): {best_site['T_CW_med']:.1f} °C")
print(f"Delta T: {best_site['delta_T']:.1f} °C")
print(f"Deterministic LCOE: {best_site['LCOE']:.2f} ct/kWh")

In [ ]:
# Run uncertainty analysis
from otex.analysis import (
    MonteCarloAnalysis, 
    UncertaintyConfig,
    TornadoAnalysis,
    plot_histogram,
    plot_tornado
)

# Configure analysis
config = UncertaintyConfig(n_samples=500, seed=42, parallel=True)

# Run Monte Carlo
mc = MonteCarloAnalysis(
    T_WW=best_site['T_WW_med'],
    T_CW=best_site['T_CW_med'],
    config=config,
    p_gross=P_GROSS,
    cost_level=COST_LEVEL
)

print("Running Monte Carlo analysis for best site...")
mc_results = mc.run(show_progress=True)

# Get statistics
stats = mc_results.compute_statistics()
lcoe_stats = stats['lcoe']

print(f"\nUncertainty Results")
print(f"=" * 40)
print(f"LCOE Mean: {lcoe_stats['lcoe_mean']:.2f} ct/kWh")
print(f"LCOE Std:  {lcoe_stats['lcoe_std']:.2f} ct/kWh")
print(f"LCOE CV:   {lcoe_stats['lcoe_cv']:.1%}")
print(f"90% CI:    [{lcoe_stats['lcoe_p5']:.2f}, {lcoe_stats['lcoe_p95']:.2f}] ct/kWh")

In [ ]:
# Run Tornado analysis
tornado = TornadoAnalysis(
    T_WW=best_site['T_WW_med'],
    T_CW=best_site['T_CW_med'],
    p_gross=P_GROSS,
    cost_level=COST_LEVEL
)

print("Running Tornado analysis...")
tornado_results = tornado.run(output='lcoe', show_progress=True)

print(f"\nParameter Sensitivity")
print(f"Baseline: {tornado_results.baseline:.2f} ct/kWh")
for name, swing in tornado_results.get_ranking()[:5]:
    print(f"  {name}: {swing:+.2f} ct/kWh")

In [ ]:
# Visualize uncertainty results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# LCOE histogram
plot_histogram(mc_results, output='lcoe', ax=axes[0], bins=40)
axes[0].set_title(f'LCOE Distribution at Best Site\n({best_site["longitude"]:.2f}, {best_site["latitude"]:.2f})', 
                  fontsize=12, fontweight='bold')

# Tornado diagram
plot_tornado(tornado_results, ax=axes[1], top_n=8)
axes[1].set_title('Parameter Sensitivity', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig(f'{REGION}_best_site_uncertainty.png', dpi=150, bbox_inches='tight')
plt.show()

## Summary

This notebook demonstrated:

1. **Region Selection**: Exploring available regions and site characteristics
2. **Analysis Execution**: Running regional OTEC analysis with CMEMS data
3. **Results Exploration**: Examining LCOE, power output, and costs across sites
4. **Visualization**: Creating maps and distribution plots
5. **Uncertainty Analysis**: Quantifying uncertainty at the best site

### Key Findings for {REGION}

- Number of feasible sites: {len(results)}
- Best LCOE: {results['LCOE'].min():.2f} ct/kWh
- LCOE range: {results['LCOE'].min():.1f} - {results['LCOE'].max():.1f} ct/kWh
- Total potential capacity: {results['p_net_nom'].sum():.0f} MW

### Next Steps

- Compare multiple regions
- Analyze different plant sizes
- Examine seasonal variations in power output